!cp '/workspace/repos/agentic-ai/PPM - MCG MADISON RIDGE DST.pdf' /workspace/data
!cp '/workspace/repos/agentic-ai/MASTER - PYTHON - SCORING MODEL - MCG MADISON RIDGE DST - v2.0.xlsx' /workspace/data
!pip3 install llama-index llama-parse llama-agents llama-index-llms-huggingface llama-index-embeddings-huggingface llama-index-llms-huggingface-api llama-index-llms-ollama llama-index-embeddings-ollama llama-index-vector-stores-neo4jvector llama-index-graph-stores-neo4j
!pip3 install openpyxl sentencepiece protobuf evaluate rouge_score absl-py tensorboardX bitsandbytes peft accelerate python-dotenv dspy-ai InstructorEmbedding
!pip3 install flash-attn --no-build-isolation
!curl -L https://ollama.com/download/ollama-linux-amd64 -o /usr/bin/ollama
!chmod +x /usr/bin/ollama
!useradd -r -s /bin/false -m -d /usr/share/ollama ollama

Install tmux, open new screen and run `ollama start`, and then `ollama pull qwen2:1.5b`

# https://www.ecfr.gov/developers/documentation/api/v1#/Versioner%20Service/get_api_versioner_v1_full__date__title__title__xml

# !cd /workspace/data && curl -X GET "https://www.ecfr.gov/api/versioner/v1/full/2024-07-23/title-17.xml?chapter=II&part=200&subpart=A&section=200.1" -H "accept: application/xml"
# !cd /workspace/data && curl -X GET "https://www.ecfr.gov/api/versioner/v1/full/2024-07-23/title-17.xml?chapter=II&part=200" -H "accept: application/xml"
!cd /workspace/data && curl -X GET "https://www.ecfr.gov/api/versioner/v1/full/2024-07-23/title-17.xml?chapter=II" -H "accept: application/xml" > title-17.xml

!cd /workspace/data && curl -X GET "https://api.finra.org/data/group/finra/name/finraRulebookMock?offset=0&limit=10000&delimiter=%2C&quoteValues=true&async=false&access_token=ae4bfc7741084e2c9944" -H  "accept: application/json" -H  "Content-Type: application/json"

!wget -O - https://debian.neo4j.com/neotechnology.gpg.key | gpg --dearmor -o /etc/apt/keyrings/neotechnology.gpg
!echo 'deb [signed-by=/etc/apt/keyrings/neotechnology.gpg] https://debian.neo4j.com stable latest' | tee -a /etc/apt/sources.list.d/neo4j.list

!apt-get update
!apt list -a neo4j
!add-apt-repository universe -y
!apt-get install neo4j=1:5.22.0 -y
!echo "neo4j-enterprise neo4j/question select I ACCEPT" | debconf-set-selections
!echo "neo4j-enterprise neo4j/license note" | debconf-set-selections
!apt remove openjdk-21-jre-headless -y
!apt install openjdk-17-jre -y

In [ ]:
# apoc plugin needs to go in /var/lib/neo4j/plugins
#### NOTE #### neo4j and apoc versions must match!!!!!!
# wget https://github.com/neo4j/apoc/releases/download/5.22.0/apoc-5.22.0-core.jar

# in vim /etc/neo4j/neo4j.conf add the following lines
# dbms.security.procedures.allowlist=apoc.*
# dbms.security.procedures.unrestricted=apoc.*

In [ ]:
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
import dspy
import numpy as np
import pandas as pd

from train_utils import get_csv_string, randomize_row_values, operators_dict, range_description_json, split_df_by_empty_columns, split_df_by_empty_rows, print_trainable_parameters


In [ ]:
import xml.etree.ElementTree as ET

# Path to your XML file
# xml_file_path = '/workspace/data/test.xml'
xml_file_path = '/workspace/data/title-17.xml'

# Parse the XML file
tree = ET.parse(xml_file_path)

# Get the root element of the XML document
root = tree.getroot()

In [ ]:


def get_tree_data(element, depth=0, list_depth=0, prev_depth=-1, lists=None, final_dict=None):
    if lists is None:
        lists = [[]]
    if final_dict is None:
        final_dict = {}

    if element.tag == "HEAD":

        if depth > list_depth:
            lists[-1].append(f"{element.text.strip()}")
        else:
            lists.append(lists[-1][:depth-2])
            lists[-1].append(f"{element.text.strip()}")
        
    elif element.tag == "P":
        # "P" indicates that there is text associated with the element
        # This only saves to the final_dict if the element has a child with tag "P"
        try:
            final_dict[str(lists[-1])] += ''.join(element.itertext())
        except:
            final_dict[str(lists[-1])] = ''.join(element.itertext())

    else:
        pass

    # Recursively call get_tree_data on each child, increasing the depth
    for child in element:
        get_tree_data(child, depth + 1, len(lists[-1])+1, depth, lists, final_dict)

    return final_dict

data = get_tree_data(root)


In [ ]:
data = {k:v for k,v in data.items() if 'CHAPTER II—SECURITIES AND EXCHANGE COMMISSION' in k}

In [ ]:

from llama_index.llms.ollama import Ollama

# model_name = "mistral:latest"
model_name = "llama3.1:latest"
# model_name = "qwen2:1.5b"
# model_name = "gemma:1.5b"

system_prompt = "You are an expert at answering questions about rules and regulations regarding Title 17—Commodity and Securities Exchanges: CHAPTER II—SECURITIES AND EXCHANGE COMMISSION. Please provide a summary of the following text, and cite any rule or regulation codes (e.g. § 230.503, Rule 501(a)) from context that support the answer:"
llm = Ollama(model=model_name, url="http://127.0.0.1:11434", system_prompt=system_prompt)
llm.metadata

model_name, num_ctx = "llama3.1", 128000
llm = dspy.OllamaLocal(model=model_name, max_new_tokens=1000, max_tokens=4000, temperature=0.0, num_ctx=num_ctx, model_type='chat')

In [ ]:
import os, dotenv
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
dotenv.load_dotenv("/workspace/repos/agentic-ai/.env")




In [ ]:
def get_metadata(s, metadata={"title":None, "chapter":None, "part":None, "subpart":None, "description":None}):
    s_split = s[2:-2].split("', '")
    key_lengths = {key: len(key) for key in metadata}
    for x in s_split:
        xsplit = x.lower().split("—")
        if len(xsplit) > 1:
            prefix = xsplit[0]
            for key, length in key_lengths.items():
                if prefix[:length] == key:
                    metadata[key] = x
                    break
    metadata["description"] = s_split[-1]
    return metadata

In [ ]:
from typing import Literal
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor

entities = Literal["RULES", "REGULATIONS"]
relations = Literal[
    "RELATED_TO",
    "REFERENCED_BY",
    "REFERS_TO",
]
all_relations = [
    "RELATED_TO",
    "REFERENCED_BY",
    "REFERS_TO",
]

validation_schema = {
    "Rules": all_relations,
    "Regulations": all_relations
}


In [ ]:
import os
import torch
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# from llama_index.embeddings.instructor import InstructorEmbedding

from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings, PropertyGraphIndex
from llama_index.readers.file import PandasExcelReader
from llama_index.core import set_global_tokenizer, Document
from llama_index.core.embeddings import resolve_embed_model
from transformers import AutoTokenizer
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)


from dotenv import load_dotenv
load_dotenv('/workspace/repos/agentic-ai/.env')

import nest_asyncio
nest_asyncio.apply()

access_token = os.getenv('HF_TOKEN')
llama_api_key = os.getenv('LLAMA_API_KEY')
persist_dir = "/workspace/data/storage/sec"

Settings.llm = llm

Settings.chunk_size = 400
Settings.chunk_overlap = 50
embed_model_name = "Alibaba-NLP/gte-Qwen2-1.5B-instruct"
# embed_model_name = "BAAI/bge-small-en-v1.5"
# embed_model_name = "hkunlp/instructor-base"
print("loading embed model...")
embed_model = HuggingFaceEmbedding(model_name=embed_model_name)
# embed_model = InstructorEmbedding(model_name=embed_model_name)

Settings.embed_model = embed_model


In [ ]:

print('loading data to vector store...')

if os.path.exists(persist_dir):
    print(f'    loading from {persist_dir}...')
    # rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
    # load index
    vector_index = load_index_from_storage(storage_context, embed_model=embed_model)
else:
    print(f'    building from scratch...')
    documents = [Document(text=t, 
                        metadata=get_metadata(m),
                        excluded_llm_metadata_keys=["title", "chapter"],) for m,t in data.items()]
    
    from llama_index.core import PropertyGraphIndex

    kg_extractor = SchemaLLMPathExtractor(
        llm=llm,
        possible_entities=entities,
        possible_relations=relations,
        kg_validation_schema=validation_schema,
        # if false, allows for values outside of the schema
        # useful for using the schema as a suggestion
        num_workers=4,
        strict=False,
    )

    # from llama_index.core.indices.property_graph import SimpleLLMPathExtractor

    # kg_extractor = SimpleLLMPathExtractor(
    #     llm=llm,
    #     max_paths_per_chunk=10,
    #     num_workers=10,
    # )

    NUMBER_OF_ARTICLES = 50

    neo_url = "bolt://localhost:7687"
    graph_store = Neo4jPropertyGraphStore(
        username="neo4j",
        password=os.getenv("NEO4J_PWD"),
        url=neo_url
    )

    vector_index = PropertyGraphIndex.from_documents(documents[:NUMBER_OF_ARTICLES], 
                                                     llm=llm,
                                                     property_graph_store=graph_store, 
                                                     embed_model=embed_model, 
                                                     kg_extractors=[kg_extractor],
                                                     show_progress=True)
    # vector_index = VectorStoreIndex.from_documents(documents, embed_model=embed_model, show_progress=True)
    vector_index.storage_context.persist(persist_dir=persist_dir)

# query_engine = vector_index.as_retriever(similarity_top_k=3)
query_engine = vector_index.as_query_engine(similarity_top_k=3)


In [ ]:
questions = [
    "If an advertisement has performance data, what disclosures are required?", 
    "What disclosure is needed in an advertisement for a government money market?",
    "What is the definition of sales literature?",
    "What is considered materially misleading?",
    "What is an accredited investor and how do they differ from an institutional buyer?"
]

gt = [
    "230.482(B)(3)(i)",
    "230.482(b)(4)(ii) or (iii). If the Government Money Market Fund that has the ability to impose fee when sold and/or temporarily suspend the ability to sell if the fund's liquidity falls below a certain level",
    "sales literature shall be deemed to include any communication (whether in writing, by radio, or by television) used by any person to offer to sell or induce the sale of securities of any investment company. Communications between issuers, underwriters and dealers are included in this definition of sales literature if such communications, or the information contained therein, can be reasonably expected to be communicated to prospective investors in the offer or sale of securities or are designed to be employed in either written or oral form in the offer or sale of securities.",
    "230.156(a)(1 and 2)",
    "Definition of Accredited investor Rule 501(a)(1-13) / Definition of Qualified Institutional Buyer Rule 230.144A(a)(I - vi)"
]

assert len(questions) == len(gt)

for idx,question in enumerate(questions):
    print("=====================================================")
    print("=====================================================")
    print(f'QUESTION {idx+1}:\n', question)
    print("=====================================================")

    answer = query_engine.query(question)
    
    print("MATT'S ANSWER:\n", gt[idx])
    print("=====================================================")
    print('ANSWER:\n', answer.response)
    print("=====================================================")
    print('CONTEXT:')
    for i,con in enumerate(answer.source_nodes):
        print(f"--Source {i}:\n", con.metadata)
        print(f"--Info {i}:\n", con.text)
        print()
        print('---')
        print()
   
    print()
    print()
    

from dspy.retrieve.llama_index_rm import LlamaIndexRM
retriever = LlamaIndexRM(query_engine)
dspy.settings.configure(lm=llm, rm=retriever)


In [ ]:
%load_ext autoreload
%autoreload 2

from models_sec import SECQnA, SECQnAReAct, ReActGenerateAnswer

sec_title17_analyzer = dspy.ReAct(ReActGenerateAnswer, tools=[dspy.Retrieve(k=3)], max_iters=3)
# sec_title17_analyzer = SECQnA(num_passages=3)

In [ ]:
questions = [
    "If an advertisement has performance data, what disclosures are required?", 
    "What disclosure is needed in an advertisement for a government money market?",
    "What is the definition of sales literature?",
    "What is considered materially misleading?",
    "What is an accredited investor and how do they differ from an institutional buyer?"
]

gt = [
    "230.482(B)(3)(i)",
    "230.482(b)(4)(ii) or (iii). If the Government Money Market Fund that has the ability to impose fee when sold and/or temporarily suspend the ability to sell if the fund's liquidity falls below a certain level",
    "sales literature shall be deemed to include any communication (whether in writing, by radio, or by television) used by any person to offer to sell or induce the sale of securities of any investment company. Communications between issuers, underwriters and dealers are included in this definition of sales literature if such communications, or the information contained therein, can be reasonably expected to be communicated to prospective investors in the offer or sale of securities or are designed to be employed in either written or oral form in the offer or sale of securities.",
    "230.156(a)(1 and 2)",
    "Definition of Accredited investor Rule 501(a)(1-13) / Definition of Qualified Institutional Buyer Rule 230.144A(a)(I - vi)"
]

assert len(questions) == len(gt)

for idx in range(len(questions)):
    print(f'QUESTION {idx+1}:\n', questions[idx])
    print("=====================================================")

    answer = sec_title17_analyzer(question=questions[idx])
    
    print("MATT'S ANSWER:\n", gt[idx])
    print("=====================================================")
    print('ANSWER:\n', answer.answer)
    print("=====================================================")
    # print('CONTEXT:')
    # for con in answer.context:
    #     print(con)
    #     print()
    #     print('---')
    #     print()
   
    print()
    print()
    break

In [ ]:
print(answer.observations)

In [ ]:
print(gt[idx])

In [ ]:
print(questions[idx])

In [ ]:
sec_title17_analyzer.inspect_history(n=3)

In [ ]:
answer.context

In [ ]:
# Rule 230.482
# Q.1 - If an advertisement has performance data, what disclosures are required?
# A.1 - 230.482(B)(3)(i)

# Q.2 - What disclosure is need in an advertisement for a government money market?
# A.2 - 230.482(b)(4)(ii) or (iii). If the Government Money Market Fund that has the ability to impose fee when sold and/or temporarily suspend the ability to sell if the fund's liquidity falls below a certain level

# Rule 230.156
# Q.1 - What is the definition of sales literature?
# A.1 - sales literature shall be deemed to include any communication (whether in writing, by radio, or by television) used by any person to offer to sell or induce the sale of securities of any investment company. Communications between issuers, underwriters and dealers are included in this definition of sales literature if such communications, or the information contained therein, can be reasonably expected to be communicated to prospective investors in the offer or sale of securities or are designed to be employed in either written or oral form in the offer or sale of securities.

# Q.2 - What is considered materially misleading?
# A.2 - 230.156(a)(1 and 2)
# Rule 230.144A and 230.501(a)

# Q.1 - What is an accredited investor and how do they differ from an institutional buyer?
# A.1 - Definition of Accredited investor Rule 501(a)(1-13) / Definition of Qualified Institutional Buyer Rule 230.144A(a)(I - vi)

# Once you play around with those, I can send some more complicated ones that cross reference other rules

In [ ]:
# dfs_str = get_csv_string(dfs)
collection = []
for value_to_extract in gt_collect:
    # if value_to_extract=="Selling Costs":
        # continue
    question = f"Extract the value for the variable name '{value_to_extract}'?"
    print(question)
    pred = finetune_program(question, verbose=True)
    print(pred.answer)
    collection.append((pred, f"{value_to_extract}: {gt_collect[value_to_extract]}"))

In [ ]:
%load_ext autoreload
%autoreload 2

from train_utils import operators_dict, range_description_json
from models_testing import SpreadSheetAnalyzer
spreadsheeet_ananlyst = SpreadSheetAnalyzer(range_description_json, operators_dict, num_passages=3)

In [ ]:
# dfs_str = get_csv_string(dfs)
collection = []
for value_to_extract in gt_collect:
    # if value_to_extract=="Selling Costs":
        # continue
    question = f"Extract the value for the variable name '{value_to_extract}'."
    print(question)
    pred = spreadsheeet_ananlyst(question, verbose=True)
    print(pred.answer)
    collection.append((pred, f"{value_to_extract}: {gt_collect[value_to_extract]}"))

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

query_engine_tools = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="value_retriever",
        description="provides useful information about a query.",
    ),
)

# query_engine_tools=[]

In [ ]:
from typing import List, Literal
from llama_index.core.bridge.pydantic import BaseModel, Field
from llama_index.core.tools import FunctionTool

def adding_values(values: List[float]):
    return sum(values)

class AddingArgs(BaseModel):
    values: List = Field(
        description="A list of values to add together."
    )

adding_tool = FunctionTool.from_defaults(
    fn=adding_values,
    name="sum_values",
    description="Add values together",
    fn_schema=AddingArgs,
)

# query_engine_tools = [adding_tool]


In [ ]:
from llama_agents import (
    AgentService,
    ToolService,
    LocalLauncher,
    MetaServiceTool,
    ControlPlaneServer,
    SimpleMessageQueue,
    AgentOrchestrator,
)

from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import ReActAgentWorker, ReActAgent

# create our multi-agent framework components
message_queue = SimpleMessageQueue()
control_plane = ControlPlaneServer(
    message_queue=message_queue,
    orchestrator=AgentOrchestrator(llm=Settings.llm),
    vector_store=vector_index.storage_context.vector_store,
    # port=8001
)

# define Tool Service
tool_service = ToolService(
    message_queue=message_queue,
    tools=[query_engine_tools],
    running=True,
    step_interval=0.5,
)

# define meta-tools here
meta_tools = [
    await MetaServiceTool.from_tool_service(
        t.metadata.name,
        message_queue=message_queue,
        tool_service=tool_service,
    )
    for t in [query_engine_tools]
]


# define Agent and agent service
worker1 = FunctionCallingAgent.from_tools(
worker1 = ReActAgentWorker.from_tools(
# worker1 = ReActAgent.from_tools(
    # [query_engine_tools],
    meta_tools,
    # tool_service,
    llm=hf_llm,
    # max_iterations=15
)
agent1 = worker1.as_agent()
# agent_server_1 = AgentService(
#     agent=agent1,
#     message_queue=message_queue,
#     description="Used to retrieve values from a spreadsheet that has been converted to a string.",
#     service_name="spreadsheet_reader_agent",
# )

In [ ]:
worker2 = ReActAgent.from_llm(llm=hf_llm)

In [ ]:
response = worker2.query("What is the value for Selling Costs? It is a value between 0 and 1.")

In [ ]:
print(response.response)

In [ ]:
launcher = LocalLauncher(
    [agent_server_1, tool_service],
    control_plane,
    message_queue,
)
query_str = "What is the Disposition Fee?"
result = launcher.launch_single(query_str)

In [ ]:
import torch
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.readers.file import PandasExcelReader
from llama_index.core import set_global_tokenizer
from llama_index.core.embeddings import resolve_embed_model
from transformers import AutoTokenizer
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

import nest_asyncio
nest_asyncio.apply()

# set_global_tokenizer(
#     AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-alpha").encode
# )



filepath = "/workspace/data/MASTER - PYTHON - SCORING MODEL - MCG MADISON RIDGE DST - v2.0.xlsx"
documents = PandasExcelReader(sheet_name="5 - Disposition Analysis").load_data(filepath)
# documents[0].text = documents[0].text.split("\n")
# import os
# os.environ['HF_TOKEN']=access_token
# model_name = "jmars/trithemius-mistral-0.3-7b"
# model_name = "Qwen/Qwen2-1.5B-Instruct"


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = "Qwen/Qwen2-1.5B-Instruct"
tokenizer_name = model_name
model = AutoModelForCausalLM.from_pretrained(model_name)
hf_llm = HuggingFaceLLM(model=model, tokenizer_name=tokenizer_name, is_chat_model=True)
Settings.llm = hf_llm


In [ ]:
from typing import List, Literal
from llama_index.core.bridge.pydantic import BaseModel, Field
from llama_index.core.tools import FunctionTool
from llama_index.core.base.llms.types import (
    ChatMessage,
    MessageRole,
)


def adding_values(values: List[float]):
    return sum(values)


class AddingArgs(BaseModel):
    values: List = Field(
        description="A list of values to add together."
    )

adding_tool = FunctionTool.from_defaults(
    fn=adding_values,
    name="sum_values",
    description="Add a list of values together",
    fn_schema=AddingArgs,
)

data=documents[0].text
usr_msg = ChatMessage(
    role=MessageRole.USER,
    # content=f"What is the sum of Disposition Fee percentage and Sales Cost percentage from this spreadsheet?\n\n##SPREADSHEET\n{data}",
    content=f"Extract the percentage values for 'Disposition Fee' and 'Sales Cost' from this spreadsheet?\n\n##SPREADSHEET\n{data}",
)

response = hf_llm.chat(
    messages=[usr_msg],
    tools=[
        adding_tool
    ],
    tool_choice="add_values",
)

print(response.message)

In [ ]:
dir(response)

In [ ]:
print(response.message)

In [ ]:
import torch
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.readers.file import PandasExcelReader
from llama_index.core import set_global_tokenizer
from llama_index.core.embeddings import resolve_embed_model
from transformers import AutoTokenizer
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

import nest_asyncio
nest_asyncio.apply()

import os
os.environ['HF_TOKEN']=access_token

# model_name = "mistralai/Mistral-7B-Instruct-v0.3"
model_name = "Qwen/Qwen2-1.5B-Instruct"
tokenizer_name = model_name
embed_model_name = "BAAI/bge-small-en-v1.5"

hf_llm = HuggingFaceLLM(model_name=model_name, tokenizer_name=tokenizer_name, is_chat_model=True, device_map='auto', max_new_tokens=2000, context_window=8000)
set_global_tokenizer(
    AutoTokenizer.from_pretrained(tokenizer_name).encode  # pass in the HuggingFace model org + repo
)
# hf_llm = HuggingFaceLLM(model_name=model_name, tokenizer_name=tokenizer_name)
# hf_llm = HuggingFaceInferenceAPI(model_name=model_name, tokenizer_name=tokenizer_name, is_chat_model=True, is_function_calling_model=True)
embed_model = HuggingFaceEmbedding(model_name=embed_model_name)

filepath = "/workspace/data/MASTER - PYTHON - SCORING MODEL - MCG MADISON RIDGE DST - v2.0.xlsx"
documents = PandasExcelReader(sheet_name="5 - Disposition Analysis").load_data(filepath)

vector_index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)
# vector_index.storage_context.persist(persist_dir="/workspace/data/storage/alpha")
query_engine = vector_index.as_query_engine(llm=hf_llm, top_k=3)

Settings.llm = hf_llm
Settings.embed_model = embed_model

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from typing import List, Literal
from llama_index.core.bridge.pydantic import BaseModel, Field
from llama_index.core.tools import FunctionTool
from llama_index.core.base.llms.types import (
    ChatMessage,
    MessageRole,
)


query_engine_tools = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="spreadsheet_value_retriever",
        description="contains the information of a spreadsheet, and is useful for retrieving specific values from a spreadsheet",
    ),
)

def adding_values(values: List[float]):
    return sum(values)


class AddingArgs(BaseModel):
    values: List = Field(
        description="A list of values to add together."
    )

adding_tool = FunctionTool.from_defaults(
    fn=adding_values,
    name="sum_values",
    description="Add a list of values together",
    fn_schema=AddingArgs,
)

In [ ]:
from llama_agents import (
    AgentService,
    ToolService,
    LocalLauncher,
    MetaServiceTool,
    ControlPlaneServer,
    SimpleMessageQueue,
    AgentOrchestrator,
)

from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import ReActAgentWorker, ReActAgent



# create our multi-agent framework components
message_queue = SimpleMessageQueue()
control_plane = ControlPlaneServer(
    message_queue=message_queue,
    orchestrator=AgentOrchestrator(llm=hf_llm),
)

# define Tool Service
tool_service = ToolService(
    message_queue=message_queue,
    tools=[query_engine_tools],#, adding_tool],
    running=True,
    step_interval=0.5,
)

# define meta-tools here
meta_tools = [
    await MetaServiceTool.from_tool_service(
        t.metadata.name,
        message_queue=message_queue,
        tool_service=tool_service,
    )
    for t in [query_engine_tools]#, adding_tool]
]


# define Agent and agent service
# worker1 = FunctionCallingAgentWorker.from_tools(
worker1 = ReActAgentWorker.from_tools(
    meta_tools,
    llm=hf_llm,
)
agent1 = worker1.as_agent()
agent_server_1 = AgentService(
    agent=agent1,
    message_queue=message_queue,
    description="Used to answer questions over Uber and Lyft 10K documents",
    service_name="uber_lyft_10k_analyst_agent",
)

In [ ]:
launcher = LocalLauncher(
    [agent_server_1, tool_service],
    control_plane,
    message_queue,
)
query_str = "What is the Disposition Fee?"
result = launcher.launch_single(query_str)

In [ ]:
from llama_index.core.agent import ReActAgentWorker, ReActAgent
agent = ReActAgent.from_tools(
    [query_engine_tools, adding_tool],
    llm=hf_llm,
    verbose=True,
    )

In [ ]:
message = ['Selling Costs',
  'Disposition Fee',
  'Net Operating Income',
  'Loan Assumption/Payoff',
  'Return of Forecasted Reserves',
  'CF Y 11',
  'Return of Maximum Offering Amount',
  'Projected Terminal Cap Rate',
  'Cash Flows']
content='Retrieve the following values from the spreadsheet: Selling Costs, Disposition Fee, Net Operating Income, Loan Assumption/Payoff, Return of Forecasted Reserves, CF Y 11, Return of Maximum Offering Amount, Projected Terminal Cap Rate, Cash Flows (categories 1 through 9)\nThen add Disposition Fee and Selling Cost together.'

usr_msg = ChatMessage(
    role=MessageRole.ASSISTANT,
    content=content
)
response = agent1.chat(content)

In [ ]:
response

In [ ]:
content='Retrieve the following values from the spreadsheet: Selling Costs, Disposition Fee, Net Operating Income, Loan Assumption/Payoff, Return of Forecasted Reserves, CF Y 11, Return of Maximum Offering Amount, Projected Terminal Cap Rate, Cash Flows (categories 1 through 9)\nThen add Disposition Fee and Selling Cost together.'

messages = [
        {"role": "user", "content": content},
    ]

In [ ]:
#  Forward NOI Growth 	2.00%
#  Selling Costs 	1.00%
#  Disposition Fee 	2.50%
	
# 	Assumes. 0-yr Hold
# 	Scenario A
# Net Operating Income	 4,644,391 
# Projected Terminal Cap Rate	5.25%
# Projected Sales Price (95%)	 88,464,592 
# Loan Assumption/Payoff	 -   
# Selling Costs	 (884,646)
# Disposition Fee	 (2,211,615)
# Return of Forecasted Reserves	 -   
# Sale Proceeds	 85,368,331 
# Proceeds from Distributions	 36,688,942 
# Return of Maximum Offering Amount	 (77,670,567)
# DST Total Gain / (Loss)	 44,386,707 

In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import logging

# create a tool
def get_the_secret_fact() -> str:
    """Returns the secret fact."""
    return "The secret fact is: A baby llama is called a 'Cria'."

tool = FunctionTool.from_defaults(fn=get_the_secret_fact)

# Define an agent
model_name = "Qwen/Qwen2-1.5B-Instruct"
llm = HuggingFaceLLM(model_name=model_name)
worker = FunctionCallingAgentWorker.from_tools([tool], llm=llm)
agent = worker.as_agent()

# Create an agent service
agent_service = AgentService(
    agent=agent,
    message_queue=message_queue,
    description="General purpose assistant",
    service_name="assistant",
)